In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import pickle as pkl

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
with open('datasets/x18_exp_l2_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)
    
print(len(graph_dict['train']) + len(graph_dict['test']))

664


### Parameters

Best param value ranges from WandB sweeps

- Cutoff distance = 11 - 12.5
- Dropout = 0.4-0.5
- Edge weights = "exp"
- Edge weight lambda = 2
- Hidden channels = 320
- Learning rate = 3.5e-5 - 4.5e-5
- Weight decay = 1e-6 - 1e-5

New best params for 18 feature model

- Dropout = 0.6
- Hidden channels = 256
- Learning rate = 4e-5
- Weight decay = 5e-6

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)


# logging params (only used for wandb metrics)
n_samples = len(graph_dict['train']) + len(graph_dict['test'])
cutoff_distance = 12

# gcn params - from best wandb sweep
# num_node_features = 18
num_node_features = 14
batch_size = 256
# hidden_channels = 240
hidden_channels = 320
dropout = 0.6

edge_weight_func = "exp"
edge_weight_lambda = 2

learning_rate = 4e-5
wd = 5e-6
epochs = 2500

In [4]:
project = 'train-final-model'

In [6]:
# # remove edge weights
# for split in graph_dict:
#     for sample in graph_dict[split]:
#         graph_dict[split][sample]['graph'].dataset[0]['edge_attr'] = None

In [5]:
graph_dict['train']['pnca_mut_0']['graph'].dataset

[Data(x=[185, 18], edge_index=[2, 5452], edge_attr=[5452, 1], y=0)]

In [6]:
# run_name = f'Run 7 - {epochs} epochs - lr {learning_rate}'
run_name = f'Only_SBMLCore_9-{epochs}epochs'

model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            cutoff_distance = cutoff_distance,
            edge_weight_func = edge_weight_func,
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = hidden_channels,
            learning_rate = learning_rate,
            wd = wd,
            dropout = dropout,
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            lambda_param= edge_weight_lambda,
            early_stop=False,
            no_node_mpfs=True,
            save_path= f'saved_models/{project}/{run_name}',
            wandb_params={
              'use_wandb': True, 
              'wandb_project': f'{project}', 
              'wandb_name': f'{run_name}',
              'n_samples': n_samples,
              'sweep': False
              }
        )

Adjusting edge index and attaching edge weights for cutoff distance 12
Removing metapredictor features from node features
Using CUDA


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dylan-home. Use `wandb login --relogin` to force relogin


Epoch: 000, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6923, Test Loss: 0.6929
Epoch: 010, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6917, Test Loss: 0.6925
Epoch: 020, Train Acc: 0.5259, Test Acc: 0.5150, Train Loss: 0.6907, Test Loss: 0.6920
Epoch: 030, Train Acc: 0.5366, Test Acc: 0.5100, Train Loss: 0.6899, Test Loss: 0.6913
Epoch: 040, Train Acc: 0.5366, Test Acc: 0.5150, Train Loss: 0.6895, Test Loss: 0.6914
Epoch: 050, Train Acc: 0.5345, Test Acc: 0.5150, Train Loss: 0.6900, Test Loss: 0.6913
Epoch: 060, Train Acc: 0.5259, Test Acc: 0.5150, Train Loss: 0.6895, Test Loss: 0.6915
Epoch: 070, Train Acc: 0.5431, Test Acc: 0.5200, Train Loss: 0.6879, Test Loss: 0.6908
Epoch: 080, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6890, Test Loss: 0.6912
Epoch: 090, Train Acc: 0.5431, Test Acc: 0.4850, Train Loss: 0.6881, Test Loss: 0.6901
Epoch: 100, Train Acc: 0.5388, Test Acc: 0.5200, Train Loss: 0.6885, Test Loss: 0.6904
Epoch: 110, Train Acc: 0.5302, Test Acc: 0.

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Test Accuracy,▁▁▂▁▁▁▂▃▄▄▅▄▆▆▆▇▇▄▇▇▇█▆▇▆▇▇▇▆█▆▆▇▇▇█▃███
Test F1,▇███▇▇█▄▅▄▄▁▅▄▇▅▅▇▇▇▇▇▇▄▃▇█▇█▆▇█▆▇▇▇▇▇▇▇
Test Loss,▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▂▂▂▂▃▂▃▂▃▅▃▅▄▄█▄▅▅
Test Sensitivity,▇███▇▇█▃▄▃▃▁▃▃▅▃▃▆▄▄▅▄▅▂▁▄▅▄▅▃▅▅▄▅▄▄▆▄▄▄
Test Specificity,▁▁▁▁▂▂▂▅▅▆▇▇▇▇▅▇▇▃▆▆▆▇▅██▆▅▆▅▇▆▅▆▅▆▇▃▇▇▇
Train Accuracy,▁▁▁▁▁▁▂▂▃▃▃▃▄▄▄▅▅▄▅▆▆▆▆▆▅▇▆▇▆▇▇▆▇▇██▆███
Train F1,▃▃▄▄▃▃▄▂▃▃▃▁▃▃▅▄▄▅▅▆▆▆▆▅▄▇▆▇▆▇▇▆▇▇█▇▆███
Train Loss,█████████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▅▄▄▃▃▃▃▃▂▂▂▂▃▁▁▁
Train Sensitivity,▇███▆▆▇▄▄▃▃▁▃▃▅▄▃▇▅▅▆▅▆▃▂▆▇▆▇▅▆▇▆▇▆▆█▆▆▆
Train Specificity,▂▁▁▁▂▂▂▅▅▅▆▇▆▆▅▇▇▄▇▇▆▇▆██▇▆▇▆█▇▆█▆▇█▅███
Test Accuracy,0.655


In [12]:
graph_dict['train']['pnca_mut_0']['graph'].dataset[0]

Data(x=[185, 18], edge_index=[2, 5452], edge_attr=[5452, 1], y=0)